<a href="https://colab.research.google.com/github/Crendal/Machine_learning/blob/main/%EA%B1%B0%EC%8B%9C%EC%A7%80%ED%91%9C%ED%81%AC%EB%A1%A4%EB%A7%81_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 시장국면 파악을 위한 거시지표 크롤링
> 보통 경제 사이클 구분할 때, 경기(경제성장)과 물가라는 2개의 변수를 기준으로 구별한다 - 삼성증권 보고서

> 경제 성장 지표와 경기 후행 지표인 물가를 같이 고려하여, 국면 분류의 유용성이 커짐.

|경기국면|경기|물가|설명|
|:---:|:---:|:---:|:---:|
|Recovery|+|-|경기상승, 물가하락|
|Expansion|+|+|경기상승, 물가상승|
|Slowdown|-|+|경기하락, 물가상승|
|Contraction|-|-|경기하락, 물가하락|

- https://ecos.bok.or.kr/api/#/DevGuide/StatisticalCodeSearch : 한국은행 통계코드 검색하는 곳

- 경기지표:보통 일반적으로, 분기 경제성장률, 경기선행지수 등 사용(분기에 나온 다는 점이 시차가 발생)
 > 월간 수출 증가율 수치의 historical 4년 z-score을 사용
 > 통계코드 : 403Y001
- 물가지표:월간 물가 상승률 수치 기반, 과거 4년 기준 z-score 사용
 > 통계코드 : 901Y009

In [ ]:
# 한국은행 ECOS api를 이용해서 필요한 정보 추출
# *AA
# 2013.01부터 월별 cpi 추출해서 res데이터 프레임에 저장
import pandas as pd
import requests
import json
from pandas.io.json import json_normalize


# 201301과 같은 형태의 '년월'이 포함된 date_list 생성
from datetime import datetime, timedelta
import time
start_date = datetime(2008, 12,1)
end_date = datetime(2023, 9, 30)

current_date = start_date
date_list = []

while current_date <= end_date:
    year = current_date.year
    month = current_date.month
    date_string = f"{year}{month:02d}"  # 월을 두 자리 숫자로 표현
    date_list.append(date_string)
    current_date = current_date.replace(day=1) + timedelta(days=32)  # 다음 달의 1일로 이동

def get_ecos(stat_code, start_date, end_date, name, cycle_type='M',guitar=""):
    url = f"http://ecos.bok.or.kr/api/StatisticSearch/QQGDEPOH62DTLE5S8O7V/json/kr/1/10/{stat_code}/{cycle_type}/{start_date}/{end_date}/{guitar}"
    response = requests.get(url)
    data = response.json()

    # ECOS api에서 필요한 년월까지 정보를 추출
    try:
        # 필요한 정보를 추출합니다.
        rows = data['StatisticSearch']['row']
        if isinstance(rows, dict):
            rows = [rows]
        # 데이터프레임에 데이터를 저장합니다.
        df = pd.DataFrame(rows)
        # 필요한 컬럼만 선택합니다.
        df = df[['TIME', 'DATA_VALUE', 'ITEM_NAME1']]
        # 컬럼 이름을 변경합니다.
        df.columns = ['date', name, '품목']
        # 'date' 컬럼을 datetime 형식으로 변환합니다.
        df['date'] = pd.to_datetime(df['date'], format='%Y%m')
        # '품목' 컬럼을 총지수로 필터링합니다.
        df = df[df['품목'] == '총지수']
        # 필요한 컬럼만 선택합니다.
        df = df[['date', name]]
        return df



    except KeyError:
        print("key error")


In [ ]:
# 소비자 물가지수 데이터 수집
res = pd.DataFrame()
for i in date_list:
  df_1 =get_ecos("901Y009",i,'202309',name='소비자물가지수')
  res = pd.concat([df_1, res],axis=0)
  time.sleep(0.1)
res.sort_values('date', ascending=True,inplace=True)
res.set_index('date',inplace=True)
res['소비자물가지수'] = res['소비자물가지수'].astype(float)
res['증감률'] = res['소비자물가지수'].pct_change()
res.dropna(inplace=True)
print(f"소비자물가지수 데이터를 {start_date}~{end_date}까지 수집해, res 데이터 프레임에 저장하였습니다")

소비자물가지수 데이터를 2008-12-01 00:00:00~2023-09-30 00:00:00까지 수집해, res 데이터 프레임에 저장하였습니다


In [ ]:
res

,소비자물가지수,증감률
date,,
2009-01-01,82.450,0.000910
2009-02-01,83.045,0.007216
2009-03-01,83.640,0.007165
2009-04-01,83.862,0.002654
2009-05-01,83.862,0.000000
...,...,...
2023-04-01,110.800,0.002171
2023-05-01,111.130,0.002978
2023-06-01,111.120,-0.000090


In [ ]:
# 수출금액지수 데이터 수집
res_1 = pd.DataFrame()
for i in date_list:
  df_2 =get_ecos("403Y001",i,'202309', name='수출금액지수')
  res_1 = pd.concat([df_2, res_1],axis=0)
  time.sleep(0.5)
res_1.sort_values('date', ascending=True,inplace=True)
res_1.set_index('date',inplace=True)
res_1['수출금액지수'] = res_1['수출금액지수'].astype(float)
res_1['증감률'] = res_1['수출금액지수'].pct_change()
res_1.dropna(inplace=True)
print(f"{'수출금액지수'} 데이터를 {start_date}~{end_date}까지 수집해, res_1 데이터 프레임에 저장하였습니다")

key error
수출금액지수 데이터를 2008-12-01 00:00:00~2023-09-30 00:00:00까지 수집해, res_1 데이터 프레임에 저장하였습니다


In [ ]:
res_1

,수출금액지수,증감률
date,,
2009-02-01,52.53,0.133823
2009-04-01,63.17,0.202551
2009-05-01,60.09,-0.048757
2009-06-01,66.26,0.102679
2009-07-01,70.67,0.066556
...,...,...
2023-04-01,118.14,-0.014679
2023-05-01,125.47,0.062045
2023-06-01,126.86,0.011078


In [ ]:
# 물가 상승률 4년 기준 z-score 구하는 함수
import numpy as np
import pandas as pd
def fouryear_z_score(df, current_time_index, window_size=48 ):
  df = pd.DataFrame(df['증감률'], index=pd.to_datetime(df.index))
  historical_inflation_rates = df[['증감률']]
  current_time_index = pd.to_datetime(current_time_index)
  row_number = df.index.get_loc(current_time_index)
  historical_inflation_rates.reset_index(inplace=True)


  # 구하고자하는 시간대에 대한 행 넘버를 구해 그 전 48개(4year)에 대한 평균과 표준 편차를 구하기
  selected_data = historical_inflation_rates.iloc[row_number-window_size:row_number]
  mean_rate = selected_data.loc[:,'증감률'].mean()
  std_deviation = selected_data.loc[:,'증감률'].std()
  current_month_rate = df.loc[current_time_index, '증감률']

# 현재 달의 z-score 계산
  z_score = (current_month_rate - mean_rate) / std_deviation

# Interpret the z-score
  if z_score > 0:
      interpretation = "+"
  elif z_score < 0:
      interpretation = "-"

  else:
      interpretation = "0"

  return z_score

In [ ]:
res.loc['2012-12-01']

소비자물가지수    92.175000
증감률         0.001782
Name: 2012-12-01 00:00:00, dtype: float64

In [ ]:
fouryear_z_score(df=res, current_time_index='2013-02-01')


0.25684475498948933


|경기국면|경기|물가|설명|그래프|
|:---:|:---:|:---:|:---:|:---:|
|Recovery|+|-|경기상승, 물가하락|-0.5|
|Expansion|+|+|경기상승, 물가상승|0.5|
|Slowdown|-|+|경기하락, 물가상승|0.5|
|Contraction|-|-|경기하락, 물가하락|-0.5|

In [ ]:
# 특정 월의 경기국면 상황 판단 함수
# time 형태 '####-##-01' 연-월 데이터
def business_cycle(time):
  a = fouryear_z_score(df=res, current_time_index=time)
  b = fouryear_z_score(df=res_1, current_time_index=time)

  if a > 0 and b < 0:
    print('Recovery')
  elif a > 0 and b > 0:
    print('Expansion')
  elif a < 0 and b > 0:
    print('Slowdown')
  else:
    print('Contraction')


In [ ]:
# 2013-01-01 부터 입력가능
business_cycle('2018-05-01')

Slowdown


In [ ]:
# 특정 월의 경기국면 상황 판단 함수
# time 형태 '####-##-01' 연-월 데이터
def business_cycle(time):
  a = fouryear_z_score(df=res, current_time_index=time)
  b = fouryear_z_score(df=res_1, current_time_index=time)

  if a > 0 and b < 0:
    print('Recovery')

  elif a > 0 and b > 0:
    print('Expansion')
  elif a < 0 and b > 0:
    print('Slowdown')
  else:
    print('Contraction')


In [ ]:
business_cycle('2022-10-01')

Recovery


In [ ]:
# 월별국면분류
from datetime import datetime, timedelta
import time
start_date = datetime(2023, 1,1)
end_date = datetime(2023, 9, 1)



In [ ]:
date_list

DatetimeIndex(['2015-01-01', '2015-02-01', '2015-03-01', '2015-04-01',
               '2015-05-01', '2015-06-01', '2015-07-01', '2015-08-01',
               '2015-09-01', '2015-10-01',
               ...
               '2022-11-01', '2022-12-01', '2023-01-01', '2023-02-01',
               '2023-03-01', '2023-04-01', '2023-05-01', '2023-06-01',
               '2023-07-01', '2023-08-01'],
              dtype='datetime64[ns]', length=104, freq=None)

In [ ]:
current_date = start_date
date_list = []
while current_date <= end_date:
    year = current_date.year
    month = current_date.month
    date_string = f"{year}{month:02d}"  # 월을 두 자리 숫자로 표현
    date_list.append(date_string)
    current_date = current_date.replace(day=1) + timedelta(days=32)  # 다음 달의 1일로 이동
date_list = [f'{date[:4]}-{date[4:6]}-01' for date in date_list]

date_list = pd.to_datetime(date_list)
table = pd.DataFrame()
date_list = pd.to_datetime(date_list)


KeyError: ignored

In [ ]:
total_cycle_list = []
for i in total_cycle:
  total_cycle_list.append()

total_cycle_list

[]

In [ ]:
# import matplotlib.pyplot as plt
# data = {
#     'Recovery': -0.5,
#     'Slowdown': 0.5,
#     'Contraction': -0.5,
#     'Expansion': 0.5
# }

# # total_cycle에 해당하는 값을 추출하여 리스트로 저장
# values = [data[word] for word in total_cycle]

# # 월별로 꺾은선 그래프 그리기
# plt.plot(values, marker='o', linestyle='-')
# plt.xticks(range(len(total_cycle)), total_cycle, rotation=90)  # x 축에 월 별 데이터를 나타내기
# plt.xlabel('Month')
# plt.ylabel('Value')
# plt.title('Economic Indicators Over Time')
# plt.grid(True)

# # 그래프 표시
# plt.show()

In [ ]:
# 환율
#달러환율
exchange_rate=get_ecos(stat_code='731Y001', start_date='20230101', end_date='20131004', name='원달러환율', cycle_type='D',guitar="0000001")

key error


In [ ]:
from datetime import datetime, timedelta
import time
start_date = datetime(2014, 1, 1)
end_date = datetime(2023, 9, 30)

current_date = start_date
date_list = []

while current_date <= end_date:
    year = current_date.year
    month = current_date.month
    day = current_date.day  # 일 추가
    date_string = f"{year}{month:02d}{day:02d}"  # 연도, 월, 일 모두 포함
    date_list.append(date_string)
    current_date = current_date.replace(day=1) + timedelta(days=32)


In [ ]:
a=pd.read_csv('/content/drive/MyDrive/NH_invest/거시변수 기간전체/data_0154_20231005.csv',encoding='cp949')
b=pd.read_csv('/content/drive/MyDrive/NH_invest/거시변수 기간전체/data_0216_20231005.csv',encoding='cp949')
c=pd.read_csv('/content/drive/MyDrive/NH_invest/거시변수 기간전체/data_0321_20231005.csv',encoding='cp949')
d=pd.read_csv('/content/drive/MyDrive/NH_invest/거시변수 기간전체/data_0336_20231005.csv',encoding='cp949')
e=pd.read_csv('/content/drive/MyDrive/NH_invest/거시변수 기간전체/data_0417_20231005.csv',encoding='cp949')
f=pd.read_csv('/content/drive/MyDrive/NH_invest/거시변수 기간전체/data_0431_20231005.csv',encoding='cp949')
g=pd.read_csv('/content/drive/MyDrive/NH_invest/거시변수 기간전체/data_0443_20231005.csv',encoding='cp949')
h=pd.read_csv('/content/drive/MyDrive/NH_invest/거시변수 기간전체/data_0458_20231005.csv',encoding='cp949')
i=pd.read_csv('/content/drive/MyDrive/NH_invest/거시변수 기간전체/data_0509_20231005.csv',encoding='cp949')
j=pd.read_csv('/content/drive/MyDrive/NH_invest/거시변수 기간전체/data_0521_20231005.csv',encoding='cp949')
k=pd.read_csv('/content/drive/MyDrive/NH_invest/거시변수 기간전체/data_0537_20231005.csv',encoding='cp949')

In [ ]:
dataframes=[a,b,c,d,e,f,g,h,i,j,k]
pd.concat

AttributeError: ignored

In [ ]:
li=pd.read_csv('/content/drive/MyDrive/NH_invest/거시변수 기간전체/제목없는 폴더/data_2112_20231005.csv',encoding='cp949')

li2=pd.read_csv('/content/drive/MyDrive/NH_invest/거시변수 기간전체/제목없는 폴더/data_2226_20231005.csv',encoding='cp949')

In [ ]:
li3=pd.read_csv('/content/drive/MyDrive/NH_invest/거시변수 기간전체/제목없는 폴더/data_2257_20231005.csv',encoding='cp949')
li4=pd.read_csv('/content/drive/MyDrive/NH_invest/거시변수 기간전체/제목없는 폴더/data_2343_20231005.csv',encoding='cp949')
li5=pd.read_csv('/content/drive/MyDrive/NH_invest/거시변수 기간전체/제목없는 폴더/data_2414_20231005.csv',encoding='cp949')
li6=pd.read_csv('/content/drive/MyDrive/NH_invest/거시변수 기간전체/제목없는 폴더/data_2438_20231005.csv',encoding='cp949')

In [ ]:
dd= pd.concat([a,b,c,d,e,f,g,h,i,j,k],axis=0)

In [ ]:
li7=pd.read_csv('/content/drive/MyDrive/NH_invest/거시변수 기간전체/제목없는 폴더/data_2939_20231005.csv', encoding='cp949')

df=pd.concat([li,li2,li3,li4,li5,li6,li7],axis=0)

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
dd.to_csv('/content/drive/MyDrive/NH_invest/거시변수 기간전체/순매수거래대금.csv')

In [ ]:
from pykrx import stock
real_get = []
for i in date_list:

  df = stock.get_market_trading_value_and_volume_by_ticker(i,i)
  df.reset_index(inplace=True)
  df= df.loc['200030']
  real_get=pd.concat([df,real_get],axis=0)
real_get

ModuleNotFoundError: ignored

In [ ]:
df = stock.get_market_trading_value_and_volume_by_ticker("20201030",'20201030')
df.reset_index(inplace=True)

<ipython-input-39-74b1790d548b>:1: DeprecationWarning: Call to deprecated function (or staticmethod) get_market_trading_value_and_volume_by_ticker. (You should use get_market_net_purchases_of_equities_by_ticker() instead) -- Deprecated since version 1.1.
  df = stock.get_market_trading_value_and_volume_by_ticker("20201030",'20201030')


In [ ]:
# 순매수 자료 네이버 증권에서 크롤링 - made by 성규씨
import requests
import pandas as pd

def get_real_stock(stock_code, page_number, name):
    url = f'https://finance.naver.com/item/frgn.naver?code={stock_code}&page={page_number}'
    table_df_list = pd.read_html(requests.get(url, headers={'User-agent': 'Mozilla/5.0'}).text)
    table_df = table_df_list[2]
    table_df.drop(table_df.index[[0, 6, 7, 8, 14, 15, 16, 22, 23, 24, 30]], inplace=True)

    kig = table_df['기관', '순매매량']
    fore = table_df['외국인', '순매매량']
    return table_df

# 기관 외국인 순매수 크롤링 및 데이터 전처리 파일에 저장
resi=pd.DataFrame()
for i in range(1,110,1):
  df =  get_real_stock('200030', i, i)
  resi = pd.concat([resi, df],axis=0)
resi.to_csv('/content/drive/MyDrive/NH_invest/데이터전처리/200030순매수.csv')

In [ ]:
resi.reset_index(drop=True)

날짜       종가    전일비     등락률      거래량       기관  외국인            
              날짜       종가    전일비     등락률      거래량     순매매량 순매매량 보유주수    보유율
0     2023.10.04  23545.0   80.0  -0.34%   1133.0    -31.0  0.0  0.0  0.00%
1     2023.09.27  23625.0  325.0  -1.36%    624.0    140.0  0.0  0.0  0.00%
2     2023.09.26  23950.0  145.0  +0.61%    355.0   -123.0  0.0  0.0  0.00%
3     2023.09.25  23805.0   75.0  -0.31%   2630.0  -2397.0  0.0  0.0  0.00%
4     2023.09.22  23880.0  335.0  -1.38%  35023.0 -30300.0  0.0  0.0  0.00%
...          ...      ...    ...     ...      ...      ...  ...  ...    ...
2175  2014.12.01  11200.0   60.0  -0.53%    775.0    217.0  0.0  0.0  0.00%
2176  2014.11.28  11260.0  105.0  +0.94%    529.0    527.0  0.0  0.0  0.00%
2177  2014.11.27  11155.0  100.0  -0.89%   1225.0    401.0  0.0  0.0  0.00%
2178  2014.11.26  11255.0    5.0  -0.04%   1920.0  -1910.0  0.0  0.0  0.00%
2179  2014.11.25  11260.0   10.0  -0.09%   1576.0   -161.0  0.0  0.0  0.00%

[2180 rows x 9 columns]

In [ ]:
resi=pd.DataFrame()
for i in range(1,3,1):
  df =  get_real_stock('200030', i, i)
  resi = pd.concat([resi, df],axis=0)


날짜       종가    전일비     등락률      거래량       기관  외국인            
            날짜       종가    전일비     등락률      거래량     순매매량 순매매량 보유주수    보유율
1   2023.10.04  23545.0   80.0  -0.34%   1133.0    -31.0  0.0  0.0  0.00%
2   2023.09.27  23625.0  325.0  -1.36%    624.0    140.0  0.0  0.0  0.00%
3   2023.09.26  23950.0  145.0  +0.61%    355.0   -123.0  0.0  0.0  0.00%
4   2023.09.25  23805.0   75.0  -0.31%   2630.0  -2397.0  0.0  0.0  0.00%
5   2023.09.22  23880.0  335.0  -1.38%  35023.0 -30300.0  0.0  0.0  0.00%
9   2023.09.21  24215.0   65.0  +0.27%    444.0    -52.0  0.0  0.0  0.00%
10  2023.09.20  24150.0   45.0  +0.19%    783.0    131.0  0.0  0.0  0.00%
11  2023.09.19  24105.0   70.0  -0.29%    734.0     33.0  0.0  0.0  0.00%
12  2023.09.18  24175.0  120.0  -0.49%   3624.0  -2009.0  0.0  0.0  0.00%
13  2023.09.15  24295.0  155.0  +0.64%   1041.0   -365.0  0.0  0.0  0.00%
17  2023.09.14  24140.0   30.0  -0.12%    690.0    -36.0  0.0  0.0  0.00%
18  2023.09.13  24170.0  135.0  -0.56%    442.0    -26.0  0.0  0.0  0.00%
19  2023.09.12  24305.0  210.0  -0.86%    318.0      6.0  0.0  0.0  0.00%
20  2023.09.11  24515.0    5.0  +0.02%  24878.0     20.0  0.0  0.0  0.00%
21  2023.09.08  24510.0  125.0  -0.51%    206.0     15.0  0.0  0.0  0.00%
25  2023.09.07  24635.0   55.0  -0.22%  36340.0    282.0  0.0  0.0  0.00%
26  2023.09.06  24690.0  325.0  -1.30%   1432.0   -368.0  0.0  0.0  0.00%
27  2023.09.05  25015.0  150.0  +0.60%    632.0    298.0  0.0  0.0  0.00%
28  2023.09.04  24865.0  200.0  +0.81%    721.0   -125.0  0.0  0.0  0.00%
29  2023.09.01  24665.0  435.0  -1.73%   1281.0   -405.0  0.0  0.0  0.00%
1   2023.08.31  25100.0  220.0  +0.88%    601.0    156.0  0.0  0.0  0.00%
2   2023.08.30  24880.0  300.0  +1.22%   1043.0    282.0  0.0  0.0  0.00%
3   2023.08.29  24580.0  150.0  +0.61%   1310.0   -182.0  0.0  0.0  0.00%
4   2023.08.28  24430.0  105.0  +0.43%   1325.0    -34.0  0.0  0.0  0.00%
5   2023.08.25  24325.0  420.0  -1.70%  13509.0  -9499.0  0.0  0.0  0.00%
9   2023.08.24  24745.0   65.0  +0.26%  20281.0 -19158.0  0.0  0.0  0.00%
10  2023.08.23  24680.0  130.0  +0.53%  49846.0 -47464.0  0.0  0.0  0.00%
11  2023.08.22  24550.0  250.0  -1.01%   1237.0   -212.0  0.0  0.0  0.00%
12  2023.08.21  24800.0  205.0  +0.83%   6459.0  -2944.0  0.0  0.0  0.00%
13  2023.08.18  24595.0  390.0  -1.56%   3469.0    532.0  0.0  0.0  0.00%
17  2023.08.17  24985.0   60.0  -0.24%   3811.0    146.0  0.0  0.0  0.00%
18  2023.08.16  25045.0  120.0  -0.48%  18403.0 -18107.0  0.0  0.0  0.00%
19  2023.08.14  25165.0  100.0  +0.40%  17594.0 -16977.0  0.0  0.0  0.00%
20  2023.08.11  25065.0   15.0  -0.06%   1878.0  -1088.0  0.0  0.0  0.00%
21  2023.08.10  25080.0   55.0  +0.22%   1149.0   -374.0  0.0  0.0  0.00%
25  2023.08.09  25025.0   10.0  -0.04%   1204.0    -11.0  0.0  0.0  0.00%
26  2023.08.08  25035.0  460.0  +1.87%  10145.0   -235.0  0.0  0.0  0.00%
27  2023.08.07  24575.0  315.0  -1.27%    759.0   -187.0  0.0  0.0  0.00%
28  2023.08.04  24890.0  130.0  +0.53%   1900.0    502.0  0.0  0.0  0.00%
29  2023.08.03  24760.0  130.0  -0.52%   2550.0    257.0  0.0  0.0  0.00%

In [ ]:
# 기관 외국인 순매수 크롤링 ! 데이터 전처리 파일에 저장! /
page = range(3,100,1)
for i in page:
 get_real_stock('200030',i,i)



In [ ]:
def get_real_stock(stock_code, page_number, name):
    url = f'https://finance.naver.com/item/frgn.naver?code={stock_code}&page={page_number}'
    table_df_list = pd.read_html(requests.get(url, headers={'User-agent': 'Mozilla/5.0'}).text)
    table_df = table_df_list[2]
    table_df.drop(table_df.index[[0, 6, 7, 8, 14, 15, 16, 22, 23, 24, 30]], inplace=True)

    kig = table_df['기관', '순매매량']
    fore = table_df['외국인', '순매매량']
    return table_df

# 기관 외국인 순매수 크롤링 및 데이터 전처리 파일에 저장
resi=pd.DataFrame()
for i in range(1,110,1):
  df =  get_real_stock('200030', i, i)
  resi = pd.concat([resi, df],axis=0)